In [ ]:
import sys, os
from os.path import join as oj


thumbnailURL = "item/%s/tiles/thumbnail?width=%s" 

class LinePrinter():
   """
   Print things to stdout on one line dynamically
   """
   def __init__(self,data):
       sys.stdout.write("\r\x1b[K"+data.__str__())
       sys.stdout.flush()

In [ ]:
def downloadImageSet( imageSet, downloadDir, girderClient, thumbSize):
    imagesDownloaded = 0
    gc=girderClient

    if not os.path.isdir(downloadDir):
        os.makedirs(downloadDir)

    for i in imageSet:
        ## I am using the tcga.barcode
        thumbName = i['tcga']['barcode']+".macro." + str(thumbSize)+".jpg"
        thumbWpath = oj(downloadDir,thumbName)
        imagesDownloaded +=1

        if not os.path.isfile(thumbWpath):
            curImage = gc.get( thumbnailURL % (i['_id'],thumbSize), jsonResp=False)
            with open(thumbWpath,"w") as fp:
                fp.write( curImage.content)
        LinePrinter("Have downloaded a total of %d images" % imagesDownloaded ) 
    print

In [ ]:
### This will download an image set that is specified by a .JSON file
import girder_client
import os,sys
import json
from os.path import join as oj

In [ ]:
useGirderDefaultPaths = True
### SURPRISE ME!!!
## rootFolder = whatever else it defaults to...

imageDataDict = {}  ## This gets the data set definition and links to download the imageData
with open("TCGA.MultiClass.MacroImageSet.json","r") as fp:
    imageDataDict = json.load(fp)

### Set Default Parameters
thumbSize = 500
testSetName  = imageDataDict['meta']['testSetName']
startDir = os.path.expanduser('~/tcgaImageSet_aadi_modelb')  ### TO DO:  Make this a parmeter

In [ ]:
## Connect to girder so I can actually download the images..
gc = girder_client.GirderClient(apiUrl=imageDataDict['serverAPIUrl'] )

In [ ]:
startDir

In [ ]:
###   Create root directory for this training set
#downloadDir = os.path.join(startDir,testSetName,'macro',str(thumbSize))
downloadDir = os.path.join(startDir)

In [ ]:
imagesDownloaded = 0 ### Going to keep a counter for the downloaded images
for lbl in imageDataDict['meta']['cohortLabels']:
	print lbl

	## FUTURE VERSIONS... can specift thumbnail size and also macro vs tiles
	trainingOutputDir = os.path.join(downloadDir,"train",lbl)
	testingOutputDir = os.path.join(downloadDir,"test",lbl)
	validationOutputDir = os.path.join(downloadDir,"val",lbl)

	downloadImageSet( imageDataDict['trainingSet'][lbl], trainingOutputDir, gc,thumbSize)
	print "Training Set Downloaded %s" % lbl
	downloadImageSet( imageDataDict['testSet'][lbl], testingOutputDir,gc,thumbSize )
	print "Testing Set Downloaded for %s" % lbl
	downloadImageSet( imageDataDict['valSet'][lbl], validationOutputDir,gc,thumbSize )
	print "Downloaded validation set for %s" % lbl